In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm
df_jo = pm.df('df_jo')
df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = df_jo.drop(columns=['index'])
#display(df_jo)

In [24]:
pd.options.display.max_columns = None
df_jo['created_at_yw'] = df_jo['created_at'].dt.strftime('%Y-%U')
fields = ['id_cr', 'id_fe','user_id','created_at','moderated_at',
          'created_at_yw', 'n_cr_fe_w','n_cr_fe_y',  #'created_at_dow',
          'transfer_type', 'stat_cr', 'stat_fe','amount','fee',
          'n_fees','n_backs','needs_m_check_recov', 'n_recovery','n_inc_back','n_inc_fees',          
          'id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment','type','recovery_status','created_at_slot'  # 'paid_at', 'to_end', 'cr_received_date'
          ]
#pm.format_to_dates(df_jo, time_format='min') # 'min','s'
df_jo = df_jo.sort_values(by=['user_id','created_at','created_at_fe','n_fees']) #   'id_cr','id_fe'
display(df_jo[df_jo.user_id == 17603  ][fields].reset_index(drop=True))

# 17144 12934 13404
# 6219	17603 8944	990215327	19878	9199	23164	

,id_cr,id_fe,user_id,created_at,moderated_at,created_at_yw,n_cr_fe_w,n_cr_fe_y,transfer_type,stat_cr,stat_fe,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_inc_back,n_inc_fees,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status,created_at_slot
0,3939,318,17603,2020-06-18 13:43:49.644285,2020-06-18 16:29:51.839464,2020-24,4,7,regular,money_back,accepted,100.0,5.0,1,1,1,1,0,1,318,2020-06-19 16:57:53.686817,2020-10-13 14:25:01.397807,Postpone Cash Request 3939,2020-08-20 23:00:34.602802,2020-08-15 16:59:19.915,58 days 03:15:30.270715,2020-07-05 22:00:00.000,2020-07-21 16:57:46.058,before,postpone,completed,13
1,3939,320,17603,2020-06-18 13:43:49.644285,2020-06-18 16:29:51.839464,2020-24,4,7,regular,money_back,accepted,100.0,5.0,2,1,1,2,0,2,320,2020-06-19 16:57:53.686817,2020-10-13 14:25:02.407455,Postpone Cash Request 3939,2020-08-20 23:00:34.602802,2020-08-15 16:59:19.915,58 days 03:15:30.270715,2020-07-21 16:57:46.058,2020-08-05 16:57:46.058,before,postpone,completed,13
2,3939,321,17603,2020-06-18 13:43:49.644285,2020-06-18 16:29:51.839464,2020-24,4,7,regular,money_back,accepted,100.0,5.0,3,1,1,3,0,3,321,2020-06-19 16:57:53.686817,2020-10-13 14:25:02.408595,Postpone Cash Request 3939,2020-08-20 23:00:34.602802,2020-08-15 16:59:19.915,58 days 03:15:30.270715,2020-08-05 16:57:46.058,2020-08-15 16:59:19.915,before,postpone,completed,13
3,3939,4525,17603,2020-06-18 13:43:49.644285,2020-06-18 16:29:51.839464,2020-24,4,7,regular,money_back,accepted,100.0,5.0,4,1,1,4,0,4,4525,2020-08-20 23:00:31.439427,2021-01-20 16:45:46.231597,rejected direct debit,2020-08-20 23:00:34.602802,2020-08-15 16:59:19.915,58 days 03:15:30.270715,NaT,NaT,after,incident,completed,13
4,13447,4548,17603,2020-08-21 06:01:05.168707,2020-08-21 08:42:14.060574,2020-33,1,7,instant,money_back,accepted,100.0,5.0,5,2,0,4,0,4,4548,2020-08-21 08:42:15.138183,2020-10-13 14:25:04.197067,Instant Payment Cash Request 13447,2020-09-08 19:26:09.692792,2020-09-07 22:00:00.000,17 days 15:58:54.831293,NaT,NaT,after,instant_payment,nice,6
5,15196,6806,17603,2020-09-08 19:28:11.840714,2020-09-09 07:21:07.268512,2020-36,1,7,instant,money_back,accepted,100.0,5.0,6,3,0,4,0,4,6806,2020-09-09 07:21:08.320383,2020-10-13 14:25:09.773048,Instant Payment Cash Request 15196,2020-10-07 19:53:53.211438,2020-10-06 22:00:00.000,28 days 02:31:48.159286,NaT,NaT,after,instant_payment,nice,19
6,19712,13336,17603,2020-10-07 19:58:59.650892,NaT,2020-40,1,7,instant,money_back,accepted,100.0,5.0,7,4,0,4,0,4,13336,2020-10-14 09:09:49.751496,2020-10-14 09:09:49.751519,Instant Payment Cash Request 19712,2020-11-05 20:13:16.184656,2020-11-05 22:00:00.000,29 days 02:01:00.349108,NaT,NaT,after,instant_payment,nice,19


In [17]:
df = pm.df('fe')
display(df[df.cash_request_id == 3759  ].reset_index(drop=True))
display(df[df.cash_request_id == 11808  ].reset_index(drop=True))


,id,cash_request_id,type,status,category,total_amount,reason,created_at,updated_at,paid_at,from_date,to_date,charge_moment
0,350,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.296759+00,NaN,2020-12-03 22:00:00+00,2020-12-18 22:00:00+00,before
1,369,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.47106+00,NaN,2020-09-04 22:00:00+00,2020-09-19 22:00:00+00,before
2,367,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.472256+00,NaN,2021-01-02 22:00:00+00,2021-01-17 22:00:00+00,before
3,366,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.473599+00,NaN,2020-08-20 22:00:00+00,2020-09-04 22:00:00+00,before
4,376,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.474888+00,NaN,2020-10-19 22:00:00+00,2020-11-03 22:00:00+00,before
5,379,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.476022+00,NaN,2020-11-03 22:00:00+00,2020-11-18 22:00:00+00,before
6,381,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.477202+00,NaN,2020-11-18 22:00:00+00,2020-12-03 22:00:00+00,before
7,371,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.478528+00,NaN,2020-09-19 22:00:00+00,2020-10-04 22:00:00+00,before
8,364,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.479808+00,NaN,2020-08-05 22:00:00+00,2020-08-20 22:00:00+00,before
9,362,3759.0,postpone,accepted,NaN,5.0,Postpone Cash Request 3759,2020-06-20 22:12:27.824285+00,2020-10-13 14:25:01.481085+00,NaN,2020-07-21 22:00:00+00,2020-08-05 22:00:00+00,before


,id,cash_request_id,type,status,category,total_amount,reason,created_at,updated_at,paid_at,from_date,to_date,charge_moment
0,6787,11808.0,postpone,cancelled,NaN,5.0,Postpone Cash Request 11808,2020-09-09 00:27:14.837617+00,2020-10-13 14:25:09.81531+00,NaN,2020-09-09 22:00:00+00,2020-10-09 22:00:00+00,after
1,6488,11808.0,postpone,cancelled,NaN,5.0,Postpone Cash Request 11808,2020-09-07 07:15:02.544904+00,2020-10-13 14:25:17.659912+00,NaN,2020-09-09 22:00:00+00,2020-10-05 10:00:00+00,after
2,6489,11808.0,postpone,cancelled,NaN,5.0,Postpone Cash Request 11808,2020-09-07 07:15:12.203507+00,2020-10-13 14:25:17.661071+00,NaN,2020-09-09 22:00:00+00,2020-10-06 10:00:00+00,after
3,6490,11808.0,postpone,cancelled,NaN,5.0,Postpone Cash Request 11808,2020-09-07 07:15:20.43333+00,2020-10-13 14:25:17.662289+00,NaN,2020-09-09 22:00:00+00,2020-10-10 10:00:00+00,after
4,6700,11808.0,postpone,cancelled,NaN,5.0,Postpone Cash Request 11808,2020-09-08 10:45:26.138108+00,2020-10-13 14:25:17.731661+00,NaN,2020-09-09 22:00:00+00,2020-10-06 10:00:00+00,after
5,6788,11808.0,postpone,cancelled,NaN,5.0,Postpone Cash Request 11808,2020-09-09 00:27:41.130073+00,2020-10-13 14:25:17.758624+00,NaN,2020-09-09 22:00:00+00,2020-09-20 10:00:00+00,after


In [3]:
pd.options.display.max_rows = None
df = pm.df('df_jo')
#df.info()

grouped_counts = df.groupby(
    ['id_cr']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
display(grouped_counts.head(10))


grouped_counts = df.groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'transfer_type', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
display(grouped_counts.head(10))

grouped_counts = df.query('stat_fe == "accepted" & stat_cr == "money_back"').groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
#display(grouped_counts.head(10))


grouped_counts = df.query('stat_fe == "accepted" & stat_cr != "money_back"').groupby(
    ['id_cr','user_id','stat_fe','stat_cr', 'type']).size().reset_index(name='counts').sort_values(
    by=['counts'], ascending=[False]).reset_index(drop=True)
grouped_counts.info()
#display(grouped_counts.head(10))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23968 entries, 0 to 23967
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id_cr   23968 non-null  int64
 1   counts  23968 non-null  int64
dtypes: int64(2)
memory usage: 374.6 KB


,id_cr,counts
0,12225,35
1,5006,28
2,4410,24
3,12452,23
4,11376,21
5,2358,19
6,11746,17
7,15319,16
8,4956,15
9,8713,15


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28481 entries, 0 to 28480
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_cr          28481 non-null  int64 
 1   user_id        28481 non-null  int64 
 2   stat_fe        28481 non-null  object
 3   stat_cr        28481 non-null  object
 4   transfer_type  28481 non-null  object
 5   type           28481 non-null  object
 6   counts         28481 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


,id_cr,user_id,stat_fe,stat_cr,transfer_type,type,counts
0,12225,16391,cancelled,money_back,instant,postpone,33
1,5006,15593,cancelled,money_back,regular,postpone,23
2,4410,99007731,cancelled,money_back,regular,postpone,22
3,11376,528,cancelled,money_back,instant,postpone,18
4,12452,23823,cancelled,money_back,instant,postpone,18
5,2358,11648,cancelled,money_back,regular,postpone,17
6,11746,21934,cancelled,money_back,instant,postpone,15
7,15319,10568,cancelled,active,instant,postpone,15
8,8713,99021706,cancelled,money_back,regular,postpone,14
9,11968,20871,cancelled,money_back,instant,postpone,14


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id_cr    298 non-null    int64 
 1   user_id  298 non-null    int64 
 2   stat_fe  298 non-null    object
 3   stat_cr  298 non-null    object
 4   type     298 non-null    object
 5   counts   298 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 14.1+ KB


In [4]:
# Pre-filtrar el dataframe para simplificar cálculos
df_filtered = df_jo[
    (df_jo['stat_fe'] == "accepted") | (df_jo['stat_cr'] == "money_back")
]

# Agregar columnas auxiliares para evitar cálculos repetitivos
df_filtered['accepted_fee'] = df_filtered.apply(
    lambda row: row['fee'] if row['stat_fe'] == "accepted" else 0, axis=1
)
df_filtered['money_back_amount'] = df_filtered.apply(
    lambda row: row['amount'] if row['stat_cr'] == "money_back" else 0, axis=1
)
df_filtered['money_back_id'] = df_filtered.apply(
    lambda row: row['id_cr'] if row['stat_cr'] == "money_back" else None, axis=1
)

# Agrupamiento optimizado
cohort_analysis_2 = df_filtered.groupby(['user_id', 'Mes_created_at'], as_index=False).agg(
    total_paid_fees=('accepted_fee', 'sum'),
    total_paid_cr=('money_back_amount', 'sum'),
    Num_Solicitudes=('money_back_id', 'nunique')
)


/tmp/ipykernel_52804/1991862846.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['accepted_fee'] = df_filtered.apply(
/tmp/ipykernel_52804/1991862846.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['money_back_amount'] = df_filtered.apply(
/tmp/ipykernel_52804/1991862846.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [5]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)

KeyboardInterrupt: 

In [ ]:
display(cohort_analysis_2.head(5))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes
0,47,2020-04,0.0,100.0,1
1,47,2020-05,5.0,10.0,1
2,47,2020-08,10.0,10.0,1
3,47,2020-09,5.0,5.0,1
4,47,2020-10,10.0,6.0,2


In [ ]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            # #total_paid_fees=('fee', 'sum'),            
            # #Num_Solicitudes=('id_cr', 'nunique')
            # #Num_Solicitudes=('id_cr', lambda x: x.unique().sum())
            # Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())

            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

In [ ]:
# Pre-filtrar el dataframe para simplificar cálculos
df_filtered = df_jo[
    (df_jo['stat_fe'] == "accepted") | (df_jo['stat_cr'] == "money_back")
]

# Agregar columnas auxiliares para evitar cálculos repetitivos
df_filtered['accepted_fee'] = df_filtered.apply(
    lambda row: row['fee'] if row['stat_fe'] == "accepted" else 0, axis=1
)
df_filtered['money_back_amount'] = df_filtered.apply(
    lambda row: row['amount'] if row['stat_cr'] == "money_back" else 0, axis=1
)
df_filtered['money_back_id'] = df_filtered.apply(
    lambda row: row['id_cr'] if row['stat_cr'] == "money_back" else None, axis=1
)

# Agrupamiento optimizado
cohort_analysis_2 = df_filtered.groupby(['user_id', 'Mes_created_at'], as_index=False).agg(
    total_paid_fees=('accepted_fee', 'sum'),
    total_paid_cr=('money_back_amount', 'sum'),
    Num_Solicitudes=('money_back_id', 'nunique')
)

# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

/tmp/ipykernel_52052/3396134713.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['accepted_fee'] = df_filtered.apply(
/tmp/ipykernel_52052/3396134713.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['money_back_amount'] = df_filtered.apply(
/tmp/ipykernel_52052/3396134713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
0,47,2020-04,0.0,100.0,1,0.000000,2020-10
1,47,2020-05,5.0,50.0,1,10.000000,2020-10
2,47,2020-08,10.0,20.0,1,50.000000,2020-10
3,47,2020-09,5.0,5.0,1,100.000000,2020-10
4,47,2020-10,10.0,6.0,2,166.666667,2020-10


In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 102105])

display(cohort_analysis_2[cohort_analysis_2.user_id == 16391])
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002])
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851])
display(cohort_analysis_2[cohort_analysis_2.user_id == 1987]) #.tail(60))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
15482,102105,2020-10,0.0,100.0,1,0.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
5421,16391,2020-06,5.0,100.0,1,5.000000,2020-10
5422,16391,2020-08,10.0,3500.0,1,0.285714,2020-10
5423,16391,2020-10,5.0,100.0,1,5.000000,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
747,2002,2020-01,0.0,100.0,1,0.00,2020-10
748,2002,2020-02,0.0,100.0,1,0.00,2020-10
749,2002,2020-03,0.0,100.0,1,0.00,2020-10
750,2002,2020-04,0.0,100.0,1,0.00,2020-10
751,2002,2020-05,0.0,100.0,1,0.00,2020-10
752,2002,2020-06,0.0,100.0,1,0.00,2020-10
753,2002,2020-07,15.0,400.0,1,3.75,2020-10
754,2002,2020-08,0.0,100.0,1,0.00,2020-10
755,2002,2020-09,0.0,100.0,1,0.00,2020-10
756,2002,2020-10,5.0,100.0,1,5.00,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
4640,13851,2020-08,5.0,550.0,2,0.909091,2020-10
4641,13851,2020-10,5.0,100.0,1,5.000000,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
733,1987,2019-12,0.0,100.0,1,0.000000,2020-10
734,1987,2020-01,0.0,100.0,1,0.000000,2020-10
735,1987,2020-02,0.0,100.0,1,0.000000,2020-10
736,1987,2020-03,0.0,100.0,1,0.000000,2020-10
737,1987,2020-04,0.0,100.0,1,0.000000,2020-10
738,1987,2020-05,0.0,100.0,1,0.000000,2020-10
739,1987,2020-06,10.0,200.0,1,5.000000,2020-10
740,1987,2020-08,15.0,300.0,1,5.000000,2020-10
741,1987,2020-10,10.0,300.0,2,3.333333,2020-10


In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 102105])

display(cohort_analysis_2[cohort_analysis_2.user_id == 16391])
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002])
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851])
display(cohort_analysis_2[cohort_analysis_2.user_id == 1987]) #.tail(60))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
15482,102105,2020-10,0.0,100.0,1,0.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
5421,16391,2020-06,5.0,100.0,1,5.000000,2020-10
5422,16391,2020-08,10.0,3500.0,1,0.285714,2020-10
5423,16391,2020-10,5.0,100.0,1,5.000000,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
747,2002,2020-01,0.0,100.0,1,0.00,2020-10
748,2002,2020-02,0.0,100.0,1,0.00,2020-10
749,2002,2020-03,0.0,100.0,1,0.00,2020-10
750,2002,2020-04,0.0,100.0,1,0.00,2020-10
751,2002,2020-05,0.0,100.0,1,0.00,2020-10
752,2002,2020-06,0.0,100.0,1,0.00,2020-10
753,2002,2020-07,15.0,400.0,1,3.75,2020-10
754,2002,2020-08,0.0,100.0,1,0.00,2020-10
755,2002,2020-09,0.0,100.0,1,0.00,2020-10
756,2002,2020-10,5.0,100.0,1,5.00,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
4640,13851,2020-08,5.0,550.0,2,0.909091,2020-10
4641,13851,2020-10,5.0,100.0,1,5.000000,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
733,1987,2019-12,0.0,100.0,1,0.000000,2020-10
734,1987,2020-01,0.0,100.0,1,0.000000,2020-10
735,1987,2020-02,0.0,100.0,1,0.000000,2020-10
736,1987,2020-03,0.0,100.0,1,0.000000,2020-10
737,1987,2020-04,0.0,100.0,1,0.000000,2020-10
738,1987,2020-05,0.0,100.0,1,0.000000,2020-10
739,1987,2020-06,10.0,200.0,1,5.000000,2020-10
740,1987,2020-08,15.0,300.0,1,5.000000,2020-10
741,1987,2020-10,10.0,300.0,2,3.333333,2020-10


In [ ]:
pd.options.display.max_rows = None
tops = df_jo[df_jo['stat_cr' ]== 'money_back'].groupby('user_id').agg(fees=('fee','sum'))
tops = df_jo[df_jo['stat_fe'] == 'accepted'  ].groupby('user_id').agg(fees=('fee','sum'))
#display(df_jo[tops])
top_users = tops.sort_values(by='fees', ascending=False).iloc[:10].reset_index()
display(top_users)
#top_users = tops.sort_values(by='fees', ascending=True).iloc[:10]
#display(top_users)

#display(df_jo[top_users])

,user_id,fees
0,17144,75.0
1,12934,55.0
2,13404,35.0
3,6219,35.0
4,17603,35.0
5,8944,35.0
6,99021532,35.0
7,1987,35.0
8,9199,35.0
9,23164,35.0


In [ ]:
df = pm.df('df_jo')
timeFormat ='S' #D
df['created_at'] = df['created_at'].dt.to_period(timeFormat) #'Min')
df['created_at_fe'] = df['created_at_fe'].dt.to_period(timeFormat) #'Min')
df['updated_at'] = df['updated_at'].dt.to_period(timeFormat) #'Min')
#df['to_receive_ini'] = df['to_receive_ini'].timedelta(seconds=math.ceil(df['to_receive_ini'].total_seconds()))
df['to_receive_ini'] = pd.to_timedelta(df['to_receive_ini']).round(timeFormat)
df['to_receive_bank'] = pd.to_timedelta(df['to_receive_bank']).round(timeFormat)
df['to_reimbur'] = pd.to_timedelta(df['to_reimbur']).round(timeFormat)
df['to_reimbur_cash'] = pd.to_timedelta(df['to_reimbur_cash']).round(timeFormat)
df['to_end'] = pd.to_timedelta(df['to_end']).round(timeFormat)
df['to_send'] = pd.to_timedelta(df['to_send']).round(timeFormat)
df['money_back_date'] = df['money_back_date'].dt.to_period(timeFormat)
df['send_at'] = df['send_at'].dt.to_period(timeFormat)
df['paid_at'] = df['paid_at'].dt.to_period(timeFormat)
df['moderated_at'] = df['moderated_at'].dt.to_period(timeFormat)
df['from_date'] = df['from_date'].dt.to_period(timeFormat)
df['to_date'] = df['to_date'].dt.to_period(timeFormat)

fields = ['id_cr','created_at','transfer_type','type','stat_cr' ,'amount','fee','n_fees','n_backs','good_user',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment' # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          #'to_receive_ini','to_receive_bank' #,'to_reimbur_cash', 'updated_at', 'to_send','send_at','moderated_at'
]

user_id = 2002# 16391 # 2002, 1987, 13851, 16391, 102105
display(cohort_analysis_2[cohort_analysis_2.user_id == user_id])

#print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-8177]): # 16391 20108, 20104, 20112,
    df_t = df[df['id_cr'] == id].sort_values(['created_at','created_at_fe']).reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[fields])

user_ids = [user_id] 
pd.options.display.max_columns = None
#print("Casos segun Cash User ID")
for id in (user_ids):
    df_t = df[(df['user_id'] == id)]#.reset_index()
    df_t = df_t[df_t['stat_cr'] == 'money_back']
    df_t = df_t[df_t['stat_fe'] == 'accepted']
    
    df_t = df_t.sort_values(['created_at','created_at_fe']).reset_index(drop=True)
    #df_t.set_index('id_cr', inplace=True)
    print(f"Only money_back - user_id {id}")
    display(df_t[fields])
    df_t = df[(df['user_id'] == id) ].sort_values(['created_at','created_at_fe']).reset_index(drop=True)

    print(f"user_id {id}")
    display(df_t[fields])

#user_ids = [13851] [2002] , 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
# vips 12934 526
# 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
# 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
# 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.


/tmp/ipykernel_52052/3401338843.py:3: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['created_at'] = df['created_at'].dt.to_period(timeFormat) #'Min')
/tmp/ipykernel_52052/3401338843.py:4: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['created_at_fe'] = df['created_at_fe'].dt.to_period(timeFormat) #'Min')
/tmp/ipykernel_52052/3401338843.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['updated_at'] = df['updated_at'].dt.to_period(timeFormat) #'Min')
/tmp/ipykernel_52052/3401338843.py:13: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['money_back_date'] = df['money_back_date'].dt.to_period(timeFormat)
/tmp/ipykernel_52052/3401338843.py:14: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['send_at'] = df['send

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,index,last_order
747,2002,2020-01,0.0,100.0,1,0.00,2020-10
748,2002,2020-02,0.0,100.0,1,0.00,2020-10
749,2002,2020-03,0.0,100.0,1,0.00,2020-10
750,2002,2020-04,0.0,100.0,1,0.00,2020-10
751,2002,2020-05,0.0,100.0,1,0.00,2020-10
752,2002,2020-06,0.0,100.0,1,0.00,2020-10
753,2002,2020-07,15.0,400.0,1,3.75,2020-10
754,2002,2020-08,0.0,100.0,1,0.00,2020-10
755,2002,2020-09,0.0,100.0,1,0.00,2020-10
756,2002,2020-10,5.0,100.0,1,5.00,2020-10


Cash Request ID: -8177


KeyError: "['good_user'] not in index"

In [ ]:
cohort_analysis_2 = (
    df_jo.groupby(['user_id', 'Mes_created_at'], as_index=False)
    .agg(
        # Sumar los valores de 'fee' donde 'stat_fe' es igual a 'accepted'
        total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
        
        # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
        total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),

        # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
        Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
    )
)

In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002]) ## 13851]) #.tail(60))

In [ ]:
# Filtrar només les files necessàries per les operacions
#filtered_df = df_jo[df_jo['stat_fe'] == 'accepted']
filtered_df = df_jo.copy()

# Agregar les dades agrupades
cohort_analysis_2 = (
    filtered_df.groupby(['user_id', 'Mes_created_at'], as_index=False)
    .agg(
        #total_paid_fees=('fee', 'sum'),
        total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
        total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
        Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique()),
        transfer_type=('transfer_type', lambda x: ', '.join(x.unique()))  # Llista valors únics
    )
)

In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))

In [ ]:
#cohort_analysis_3 = filtered_df.groupby(['user_id', 'Mes_created_at'])
#display(cohort_analysis_3)

In [ ]:
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr']
) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['index'] = cohort_analysis_2['index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

# Resultado final del DataFrame de análisis de cohortes
#cohort_analysis_2
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))

In [ ]:
#cohort_analysis_2.info
#cohort_analysis_2.count
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851]) #.tail(60))


In [ ]:
# Crear columna 'Cohorte' que indica el mes de la primera solicitud de cada cliente
df_jo['Cohorte'] = df_jo.groupby('user_id')['created_at'].transform('min').dt.to_period('M')
#df_jo.head(10)

In [ ]:
# Agrupar los datos por cohorte y mes para calcular las métricas mensuales
cohort_analysis = df_jo.groupby(['Cohorte', 'Mes_created_at']).agg(
    Num_Clientes=('user_id', 'nunique'),  # Número de clientes únicos por cohorte y mes
    Num_Solicitudes=('id_cr', 'nunique'),  # Total de solicitudes únicas por cohorte y mes
    Solicitudes_Instant=('transfer_type', lambda x: (x == 'instant').sum()),  # Solicitudes tipo 'instant' por cohorte y mes
    Solicitudes_Regular=('transfer_type', lambda x: (x == 'regular').sum()),  # Solicitudes tipo 'regular' por cohorte y mes
    Clientes_con_fees=('fee', lambda x: (x > 0).sum()),  # Clientes con fees por cohorte y mes
    CR=('id_cr', lambda ids: df_jo[df_jo['id_cr'].isin(ids) & df_jo['cash_request_received_date'].notnull()]['amount'].sum()),

    Fees=('fee', 'sum'),  # Total de fees por cohorte y mes
    # Contar las líneas con 'stat_fe' == 'accepted'
    paid_fees=('stat_fe', lambda x: (x == 'accepted').sum()),  
    # Contar las líneas con 'stat_fe' != 'accepted'
    non_paid_fees=('stat_fe', lambda x: (x != 'accepted').sum()),
    # Sumar 'total_amount' cuando 'stat_fe' == 'accepted'
    total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),
    # Sumar 'total_amount' cuando 'stat_fe' != 'accepted'
    total_non_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] != 'accepted'].sum()),
     # Contar las líneas con 'stat_cr' == 'money_back'
    paid_cr=('stat_cr', lambda x: (x == 'money_back').sum()),  
    # Contar las líneas con 'stat_fe' != 'accepted'
    non_paid_cr=('stat_cr', lambda x: (x != 'money_back').sum()),
    # Sumar 'total_amount' cuando 'stat_fe' == 'accepted'
    total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].sum()),
    # Sumar 'total_amount' cuando 'stat_fe' != 'accepted'
    total_non_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] != 'money_back'].sum()),
    # Métrica de "credit lapse" (diferencia entre reimbursement_date y created_at)
    credit_lapse=('reimbursement_date', lambda x: (x - df_jo.loc[x.index, 'created_at']).dt.days.mean()),
    # Métrica de "credit payment lapse" (diferencia entre money_back_date y reimbursement_date)
    credit_payment_lapse=('money_back_date', lambda x: (x - df_jo.loc[x.index, 'reimbursement_date']).dt.days.mean())

).reset_index()

# Calcular los porcentajes de non_paid_fees
cohort_analysis['paid_fees_percent_qty'] = (cohort_analysis['paid_fees'] / (cohort_analysis['paid_fees'] + cohort_analysis['non_paid_fees']))
cohort_analysis['paid_fees_percent_amount'] = (cohort_analysis['total_paid_fees'] / cohort_analysis['Fees'])   # % en monto
cohort_analysis['paid_cr_percent_qty'] = (cohort_analysis['paid_cr'] / (cohort_analysis['paid_cr'] + cohort_analysis['non_paid_cr']))
cohort_analysis['paid_cr_percent_amount'] = (cohort_analysis['total_paid_cr'] / cohort_analysis['CR'])   # % en monto
cohort_analysis['index'] = (cohort_analysis['total_paid_fees'] / cohort_analysis['total_paid_cr'])   # % index

# Eliminar las columnas de pago y no pago
cohort_sizes_monthly = cohort_analysis.drop(
    ['paid_fees', 'non_paid_fees', 'total_paid_fees', 'total_non_paid_fees','paid_cr', 'non_paid_cr', 'total_paid_cr', 'total_non_paid_cr'], axis=1, errors='ignore')

# Mostrar los resultados de las cohortes por mes
#cohort_analysis

In [ ]:
cohort_analysis
#display(cohort_analysis[cohort_analysis.user_id == 2002])

In [ ]:
df = pm.df('df_jo')
#df.info()

cr_id = ['id_cr','id_fe','user_id', 'created_at','created_at_fe','amount','fee','stat_cr','stat_fe','transfer_type','type',
            'to_receive_ini', 'to_receive_bank','to_reimbur','to_reimbur_cash','to_end','to_send',
             'send_at', 'cr_received_date', 'money_back_date', 'reimbursement_date',
            'paid_at','charge_moment','moderated_at','reason','category','from_date','to_date', 'recovery_status'] # ,'id_y','from_date','to_date','reason','recovery_status', 'cash_request_id'

print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-18264]): # 16391 20108, 20104, 20112,
    df_t = df[df['id_cr'] == id].reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[cr_id].sort_values('created_at'))

user_ids = [54879, 9900458, 2002, 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
# vips 12934 526
# 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
# 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
# 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.

pd.options.display.max_columns = None
print("Casos segun Cash User ID")
for id in (user_ids):
    #df_t = df[df['user_id'] == id].reset_index()
    df_t = df[(df['user_id'] == id) & (df['stat_cr'] == 'money_back')].reset_index()
    print(f"User ID: {id}")
    display(df_t[cr_id].sort_values('created_at').reset_index())